In [1]:
import pandas as pd

In [241]:
general = pd.read_csv('general_success.csv')
#organizer = pd.read_csv('organizer_dollar.csv')
comments = pd.read_csv('comments_dollar.csv')
updates = pd.read_csv('updates_dollar.csv')
#donations = pd.read_csv('donations_dollar.csv')

In [242]:
#컬럼삭제
general = general.drop(['Unnamed: 0','total_comments','success','dum_success'],axis=1)
general.shape

(3904, 35)

In [243]:
#컬럼명 수정
general = general.rename(index=str, columns={"tags": "category", "Char_age": "char_age"})
general.columns

Index(['chid', 'url', 'scrape_date', 'title', 'created_date', 'funds_raised',
       'funds_target', 'picture', 'org_picture', 'counts_status_code',
       'velocity_status_code', 'updates_status_code', 'comments_status_code',
       'donations_status_code', 'currency', 'category', 'story',
       'receiver_name', 'receiver_tax', 'total_photos', 'total_co_photos',
       'total_community_photos', 'total_updates', 'total_donations',
       'total_unique_donors', 'amount_raised_unattributed',
       'number_of_donations_unattributed', 'campaign_hearts',
       'social_share_total', 'num_recent_donations', 'recent_time_window',
       'funds_raised_dollar', 'funds_target_dollar',
       'amount_raised_unattributed_dollar', 'char_age'],
      dtype='object')

In [244]:
# success 컬럼 및 더미 변수 만들기
# 모금금액이 목표금액을 넘기면 True(1), 못넘기면 False(0)
general = general.assign(success = general['funds_raised'] >= general['funds_target'])
general['dum_success']=pd.get_dummies(general['success'])[True]
general

,chid,url,scrape_date,title,created_date,funds_raised,funds_target,picture,org_picture,counts_status_code,...,campaign_hearts,social_share_total,num_recent_donations,recent_time_window,funds_raised_dollar,funds_target_dollar,amount_raised_unattributed_dollar,char_age,success,dum_success
0,1.0,https://www.gofundme.com/f/-chris-beaty-memori...,2020-11-03,Chris Beaty Memorial Fund,2020-06-03,175083.0,200000.0,True,False,200,...,1691.0,9069.0,0.0,48 hours,175083,200000.0,0.0,153,False,0
1,2.0,https://www.gofundme.com/f/1-prosent-chance,2020-11-03,1 percent chance,2020-06-21,299042.0,2000000.0,True,False,200,...,542.0,1882.0,0.0,48 hours,33792,226000.0,0.0,135,False,0
2,3.0,https://www.gofundme.com/f/100-computers-in-10...,2020-11-04,MELS needs #100ComputersIn100Days,2020-06-19,13845.0,30000.0,True,True,200,...,119.0,383.0,0.0,48 hours,13845,30000.0,8196.0,138,False,0
3,4.0,https://www.gofundme.com/f/100-to-victims-of-t...,2020-11-03,100% to Victims of Taal Volcano in Philippines,2020-01-16,25.0,5000.0,True,True,200,...,1.0,0.0,0.0,48 hours,25,5000.0,0.0,292,False,0
4,5.0,https://www.gofundme.com/f/100k-for-center-for...,2020-11-04,$100k for Center for Family Life,2020-11-02,13950.0,100000.0,True,False,200,...,17.0,0.0,16.0,48 hours,13950,100000.0,1100.0,2,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,3901.0,https://www.gofundme.com/f/zuayzb-helping-the-...,2020-11-03,Helping the Hansens: A Family in Crisis,2020-07-24,68691.0,15000.0,True,False,200,...,636.0,1574.0,1.0,48 hours,68691,15000.0,48000.0,102,True,1
3900,3902.0,https://www.gofundme.com/f/zurn-strong,2020-11-03,Zurn Strong,2020-10-13,41330.0,20000.0,True,False,200,...,348.0,828.0,1.0,48 hours,41330,20000.0,41330.0,21,True,1
3901,3903.0,https://www.gofundme.com/f/zw3yb-culver-family,2020-11-03,Sean Culver recovery and rehab fund,2020-08-28,59205.0,50000.0,True,False,200,...,559.0,929.0,0.0,48 hours,59205,50000.0,0.0,67,True,1
3902,3904.0,https://www.gofundme.com/f/zwqzz9-50000,2020-11-03,In loving memory of Koorosh (Cyrus) Yaghoubian,2020-08-09,58767.0,80000.0,True,True,200,...,298.0,699.0,0.0,48 hours,58767,80000.0,0.0,86,False,0


In [245]:
# 카테고리 그룹화 컬럼 만들기
general = general.assign(group_category = general['category'])

In [246]:
def replace(entry):
    
    string = str(entry)
    if string.startswith('Medical'):
        return 'Medical, Illness & Healing'
    elif string.startswith('Accidents'): 
        return 'Accidents & Emergencies'
    elif string.startswith('Funerals'): 
        return 'Funerals & Memorials'
    else:
        return 'Others'


general['group_category'] = general['group_category'].apply(replace)
general['group_category'].value_counts()

Medical, Illness & Healing    1117
Accidents & Emergencies       1104
Funerals & Memorials          1030
Others                         653
Name: group_category, dtype: int64

In [247]:
# 실제 category별 카운트 가져와서 위와 비교해보기 (잘되었는지)
general['category'].value_counts()

Medical, Illness & Healing     1117
Accidents & Emergencies        1104
Funerals & Memorials           1030
Community & Neighbors           193
Education & Learning             97
Other                            72
Volunteer & Service              58
Animals & Pets                   52
Missions, Faith & Church         48
Creative Arts, Music & Film      36
Sports, Teams & Clubs            32
Babies, Kids & Family            26
Business & Entrepreneurs         12
Environment                       9
Dreams, Hopes & Wishes            7
Celebrations & Events             5
Competitions & Pageants           2
Non-Profits & Charities           2
Rent, Food & Monthly Bills        1
Travel & Adventure                1
Name: category, dtype: int64

In [248]:
total_comments = comments.groupby('chid').agg({"comment_id":"count"})
total_comments = total_comments.rename(index=str, columns={"comment_id": "total_comments"})
total_comments = total_comments.reset_index()
total_comments = total_comments.astype({"chid":"float64"})
general = general.merge(total_comments, on='chid', how='left')

In [249]:
general['total_comments'].fillna(0, inplace = True)

In [250]:
total_updates = updates.groupby('chid').agg({"update_id":"count"})
total_updates = total_updates.rename(index=str, columns={"update_id": "update_counts"})
total_updates = total_updates.reset_index()
total_updates = total_updates.astype({"chid":"float64"})
general = general.merge(total_updates, on='chid', how='left')

In [251]:
general['update_counts'].fillna(0, inplace = True)

In [252]:
general

,chid,url,scrape_date,title,created_date,funds_raised,funds_target,picture,org_picture,counts_status_code,...,recent_time_window,funds_raised_dollar,funds_target_dollar,amount_raised_unattributed_dollar,char_age,success,dum_success,group_category,total_comments,update_counts
0,1.0,https://www.gofundme.com/f/-chris-beaty-memori...,2020-11-03,Chris Beaty Memorial Fund,2020-06-03,175083.0,200000.0,True,False,200,...,48 hours,175083,200000.0,0.0,153,False,0,Funerals & Memorials,74.0,1.0
1,2.0,https://www.gofundme.com/f/1-prosent-chance,2020-11-03,1 percent chance,2020-06-21,299042.0,2000000.0,True,False,200,...,48 hours,33792,226000.0,0.0,135,False,0,"Medical, Illness & Healing",0.0,0.0
2,3.0,https://www.gofundme.com/f/100-computers-in-10...,2020-11-04,MELS needs #100ComputersIn100Days,2020-06-19,13845.0,30000.0,True,True,200,...,48 hours,13845,30000.0,8196.0,138,False,0,Others,7.0,4.0
3,4.0,https://www.gofundme.com/f/100-to-victims-of-t...,2020-11-03,100% to Victims of Taal Volcano in Philippines,2020-01-16,25.0,5000.0,True,True,200,...,48 hours,25,5000.0,0.0,292,False,0,Accidents & Emergencies,0.0,0.0
4,5.0,https://www.gofundme.com/f/100k-for-center-for...,2020-11-04,$100k for Center for Family Life,2020-11-02,13950.0,100000.0,True,False,200,...,48 hours,13950,100000.0,1100.0,2,False,0,Others,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,3901.0,https://www.gofundme.com/f/zuayzb-helping-the-...,2020-11-03,Helping the Hansens: A Family in Crisis,2020-07-24,68691.0,15000.0,True,False,200,...,48 hours,68691,15000.0,48000.0,102,True,1,"Medical, Illness & Healing",17.0,0.0
3900,3902.0,https://www.gofundme.com/f/zurn-strong,2020-11-03,Zurn Strong,2020-10-13,41330.0,20000.0,True,False,200,...,48 hours,41330,20000.0,41330.0,21,True,1,Funerals & Memorials,27.0,0.0
3901,3903.0,https://www.gofundme.com/f/zw3yb-culver-family,2020-11-03,Sean Culver recovery and rehab fund,2020-08-28,59205.0,50000.0,True,False,200,...,48 hours,59205,50000.0,0.0,67,True,1,"Medical, Illness & Healing",41.0,3.0
3902,3904.0,https://www.gofundme.com/f/zwqzz9-50000,2020-11-03,In loving memory of Koorosh (Cyrus) Yaghoubian,2020-08-09,58767.0,80000.0,True,True,200,...,48 hours,58767,80000.0,0.0,86,False,0,Funerals & Memorials,1.0,0.0


In [265]:
general = general.astype({"chid":"int64", "dum_success":"int64"})

In [266]:
general.to_csv('general_final.csv', header= True ,index= False)

In [267]:
general.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3904 entries, 0 to 3903
Data columns (total 40 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   chid                               3904 non-null   int64  
 1   url                                3904 non-null   object 
 2   scrape_date                        3904 non-null   object 
 3   title                              3904 non-null   object 
 4   created_date                       3904 non-null   object 
 5   funds_raised                       3904 non-null   float64
 6   funds_target                       3853 non-null   float64
 7   picture                            3904 non-null   bool   
 8   org_picture                        3904 non-null   bool   
 9   counts_status_code                 3904 non-null   int64  
 10  velocity_status_code               3904 non-null   int64  
 11  updates_status_code                3904 non-null   int64